<a href="https://colab.research.google.com/github/jwengr/dacon/blob/main/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%20%EC%82%AC%EC%9A%A9%EC%9E%90%20%EC%97%B0%EC%B2%B4%20%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/TabNET%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-tabnet

In [ ]:
import pandas as pd
import numpy as np

from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
PATH = 'drive/My Drive/dacon/credit'

In [ ]:
train_df = pd.read_csv(PATH+'/dataset/train.csv').drop('index',axis=1)

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 849kB 16.9MB/s 
     |████████████████████████████████| 276kB 42.0MB/s 
     |████████████████████████████████| 112kB 39.9MB/s 
     |████████████████████████████████| 184kB 43.0MB/s 
     |████████████████████████████████| 829kB 37.8MB/s 
     |████████████████████████████████| 1.3MB 33.8MB/s 
     |████████████████████████████████| 296kB 54.9MB/s 
     |████████████████████████████████| 143kB 43.6MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=7ced3a740184b77ed13d60a13b3ad35f39a523161d7a6bcee75186ad92c3e7b6
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=5cf8bbb8851498909fc34fb745847c2ba11fa6538a8038aa7e74e85f3893673e
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAM

In [ ]:
import pytorch_lightning as pl
import torch
from torch import nn

In [ ]:
class GhostBN(nn.Module):
    def __init__(self,inp_dim,vbs):
        super().__init__()
        self.bn = nn.BatchNorm1d(inp_dim)
        self.vbs = vbs
        
    def forward(self,x):
        chunks = torch.chunk(x,x.size(0)//self.vbs)
        x = torch.cat([self.bn(chunk) for chunk in chunks])
        return x

In [ ]:
class GLU(nn.Module):
    def __init__(self,inp_dim):
        super().__init__()
        self.fc = nn.Linear(inp_dim,inp_dim*2)
    
    def forward(self,x):
        x = self.fc(x)
        x = x[:inp_dim]*torch.sigmoid(x[inp_dim:])
        return x

In [ ]:
class DecisionStep(nn.Module):
    def __init__(self,inp_dim,vbs):
        super().__init__()
        self.fc = nn.Linear(inp_dim,inp_dim)
        self.gbn = GhostBN(inp_dim,vbs)
        self.glu = GLU(inp_dim)
    
    def forward(self,x):
        x = self.fc(x)
        x = self.gbn(x)
        x = self.glu(x)
        return x

In [ ]:
class SharedBlock(nn.Module):
    def __init__(self,inp_dim,vbs,n=2):
        super().__init__()
        self.steps = nn.ModuleList()
        for _ in range(n) : self.steps.append(DecisionStep(inp_dim,vbs))

    def forward(self,x):
        x = self.steps[0](x)
        for i in range(1,n): x = (self.steps[i](x)+x)/0.5**2
        return x

In [ ]:
class DecisionDependentBlock(nn.Module):
    def __init__(self,inp_dim,vbs,n=2):
        super().__init__()
        self.steps = nn.ModuleList()
        for _ in range(n) : self.steps.append(DecisionStep(inp_dim,vbs))
    
    def forward(self,x):
        for i in range(n): x = (self.steps[i](x)+x)/0.5**2
        return x

In [ ]:
class FeatureTransformer(nn.Module):
    def __init__(self,inp_dim,vbs,n_1=2,n_2=2):
        super().__init__()
        self.shared_block = SharedBlock(inp_dim,vbs,n=n_1)
        self.descision_dependent_block = DecisionDependentBlock(inp_dim,vbs,n=n_2)

    def forward(self,x):
        x = shared_block(x)
        x = descision_dependent_block(x)
        return x

In [ ]:
class Sparsemax(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        x_sorted,_ = torch.sort(z,descending=True)
        x_cumsum = torch.cumsum(x_sorted,0)
        k_arr = torch.range(1,x.size(0))
        temp_arr = 1+k_arr*x_sorted
        k = torch.argmax((temp_arr>x_cumsum)*k_arr)+1
        threshold = (x_cumsum[k-1]-1)/k
        temp_x = x-threshold
        return temp_x*(temp_x>0)


In [ ]:
class AttentionTransformer(nn.Module):
    def __init__(self,inp_dim,vbs,n=2):
        super().__init__()
        self.fc = nn.Linear(inp_dim,inp_dim)
        self.gbn = GhostBN(inp_dim,vbs)
        self.smax = Sparsemax()

    def forward(self,x):
        x = self.fc(x)
        x = self.gbn(x)
        x = x*self.smax(x)
        return x

In [ ]:
class TabNet(pl.LightningModule):

    def __init__(self,inp_dim,vbs,):
        super().__init__()
        self.feature_transformers = nn.ModuleList()
        self.feature_transformers

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        embedding = self.encoder(x)
        return embedding

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop. It is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer